# Interactive OptVL Aircraft Analysis

### Introduction
This notebook provides an interactive example of using OptVL, an aerodynamic analysis tool based on AVL (Athena Vortex Lattice). We will explore how basic geometric parameters of a wing affect its lift distribution. You'll be able to use sliders to change the wing's chord, twist, and span (which influences aspect ratio) and see the aerodynamic results updated in real-time.

This example is aimed at senior undergraduate students in Aerospace Engineering. Some familiarity with basic aerodynamic concepts is assumed, but extensive Python knowledge is not required to run and interact with the notebook.

**Key Objectives:**
- Demonstrate loading an aircraft geometry into OptVL.
- Show how to modify geometric parameters programmatically.
- Illustrate running an aerodynamic analysis.
- Visualize results (lift distribution and geometry) interactively using Matplotlib and ipywidgets.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from IPython.display import display
from optvl import OVLSolver

## Load Aircraft Geometry
We will start by loading a simple rectangular wing geometry defined in an AVL file.
Make sure `rectangle.avl` is in the same directory as this notebook.

In [ ]:
try:
    ovl = OVLSolver(geo_file="rectangle.avl")
    print("AVL file loaded successfully.")
    surface_names = ovl.get_surface_names()
    print("Available surfaces:", surface_names)
    if surface_names:
        main_wing_name = surface_names[0] # Assume the first surface is the main wing
        print(f"Assuming '{main_wing_name}' as the main wing.")
    else:
        main_wing_name = None
        print("Error: No surfaces found in the AVL file.")
except FileNotFoundError:
    print("Error: rectangle.avl not found. Make sure it's in the same directory as the notebook.")
    ovl = None
    main_wing_name = None
except Exception as e:
    print(f"An error occurred during OVLSolver initialization: {e}")
    ovl = None
    main_wing_name = None

### Basic Aerodynamic Concepts
Before we dive into modifying the geometry, let's briefly review the key parameters we'll be adjusting:

-   **Chord:** The length of the wing section from the leading edge to the trailing edge. Changing the chord directly affects the wing area and thus the lift generated.
-   **Twist (Angle of Incidence):** The angle at which a wing section is set relative to a reference plane (often the root chord's angle). Geometric twist refers to a variation in the angle of incidence along the span. Twisting a wing (e.g., "washout," where the tip is at a lower angle of incidence than the root) can alter the spanwise lift distribution, potentially reducing lift at the tips to delay stall or modify induced drag.
-   **Span & Aspect Ratio (AR):**
    -   **Span (b):** The distance from one wingtip to the other.
    -   **Aspect Ratio (AR):** A measure of how long and slender a wing is. It's defined as AR = b^2 / S, where S is the wing area. For a rectangular wing, AR = b / c (where c is the chord). High aspect ratio wings (like on gliders) are generally more aerodynamically efficient at subsonic speeds (lower induced drag), while low aspect ratio wings are common on fighter jets. In this notebook, changing the "Span Factor" will directly alter the span, and consequently, the aspect ratio.

The **lift distribution** shows how the lift is distributed along the wingspan. An elliptical lift distribution is theoretically optimal for minimizing induced drag on an untwisted wing. Modifying chord and twist will change the shape of this distribution.

## Helper Functions for Geometry Modification
These functions will allow us to interactively change the wing's chord, twist, and span (affecting aspect ratio).
We'll store the original geometry parameters first.

In [ ]:
if ovl and main_wing_name:
    try:
        original_surface_params = ovl.get_surface_params(surf_name=main_wing_name, include_geom=True)
        original_chords = original_surface_params[main_wing_name]['chords']
        original_aincs = original_surface_params[main_wing_name]['aincs']
        original_yles = original_surface_params[main_wing_name]['yles']
        
        # Calculate original span (assuming yles are sorted and define half-span if symmetric)
        # For a simple wing like rectangle.avl, yles might be [0, span/2] or similar
        if len(original_yles) > 1:
            # Assuming yles define section boundaries, max(yles) could be tip y-location
            # For a wing defined from root to tip, span is 2 * max(yles) if symmetric,
            # or max(yles) - min(yles) if it's a full definition.
            # For rectangle.avl, it's often just one section defining the half-span.
            # Let's get it directly from Bref if Sref, Cref, Bref are well defined for a rectangle.
            ref_data = ovl.get_reference_data()
            original_bref = ref_data.get('Bref')
            original_sref = ref_data.get('Sref')
            original_cref = ref_data.get('Cref')

            if original_bref:
                original_span = original_bref
                print(f"Original span (Bref): {original_span}")
            else: # Fallback: infer from yles (may need adjustment based on AVL structure)
                # This logic might need refinement based on how rectangle.avl is structured
                min_y = np.min(original_yles)
                max_y = np.max(original_yles)
                # Check if yduplicate is used
                y_duplicate_val = ovl.get_surface_param(main_wing_name, 'yduplicate')
                if y_duplicate_val is not None and y_duplicate_val != 0.0 : # typical yduplicate is 0 for none, 1 for y-sym, -1 for y-anti-sym
                    original_span = 2 * max_y # Assumes max_y is the tip of one side of a symmetric wing
                else:
                    original_span = max_y - min_y # Full span defined
                print(f"Inferred original span from yles: {original_span}")


            if original_cref:
                 original_root_chord = original_cref # For a rectangle, Cref is usually the chord
                 print(f"Original root chord (Cref): {original_root_chord}")
            elif len(original_chords)>0 :
                 original_root_chord = original_chords[0]
                 print(f"Original root chord (from section): {original_root_chord}")
            else:
                original_root_chord = 1.0 # Fallback
                print("Warning: Could not determine original root chord, defaulting to 1.0")

        else: # Single section wing
            original_span = 2 * original_yles[0] if original_yles[0] > 0 else 1.0 # Educated guess for rectangle.avl style
            original_root_chord = original_chords[0] if len(original_chords) > 0 else 1.0
            print(f"Original span (single section guess): {original_span}")
            print(f"Original root chord (single section): {original_root_chord}")

        print(f"Original Chords: {original_chords}")
        print(f"Original Twists (aincs): {original_aincs}")
        print(f"Original YLEs: {original_yles}")

    except Exception as e:
        print(f"Error getting original surface parameters: {e}")
        original_chords = np.array([1.0]) # Fallback
        original_aincs = np.array([0.0])  # Fallback
        original_yles = np.array([0.0, 1.0]) # Fallback
        original_span = 2.0 # Fallback
        original_root_chord = 1.0 # Fallback
else:
    print("OptVL not loaded or main wing not identified. Cannot retrieve original geometry.")
    # Define fallbacks so notebook can still be "run" partially for structure checking
    original_chords = np.array([1.0])
    original_aincs = np.array([0.0])
    original_yles = np.array([0.0, 1.0])
    original_span = 2.0
    original_root_chord = 1.0
    main_wing_name = "Wing" # fallback

In [ ]:
def update_chord(ovl_solver, wing_name, chord_factor, initial_chords):
    if ovl_solver and wing_name:
        new_chords = initial_chords * chord_factor
        try:
            ovl_solver.set_surface_param(wing_name, "chords", new_chords, update_geom=True)
            # print(f"Updated chords: {new_chords}")
            # Update Cref and Sref if they are used by OptVL for non-dim
            ref_data = ovl_solver.get_reference_data()
            if 'Cref' in ref_data:
                ref_data['Cref'] = new_chords[0] # Assuming root chord for Cref
            if 'Sref' in ref_data and 'Bref' in ref_data: # Assuming rectangular, Sref = Cref * Bref
                 ref_data['Sref'] = new_chords[0] * ref_data['Bref']
            ovl_solver.set_reference_data(ref_data)

        except Exception as e:
            print(f"Error updating chord: {e}")

def update_twist(ovl_solver, wing_name, tip_twist_deg, root_twist_deg, initial_aincs, section_yles, wing_total_span):
    if ovl_solver and wing_name:
        num_sections = len(section_yles)
        new_aincs = np.copy(initial_aincs) # Start with original aincs

        if num_sections > 1 and wing_total_span > 1e-6: # Avoid division by zero for zero span
            # Apply linear twist from root_twist_deg to tip_twist_deg
            # This assumes section_yles are spanwise positions from the root (or a reference point)
            # For a wing defined from y=0 to y=span/2, y_norm goes from 0 to 1.
            
            # Find min and max y values to normalize current section positions
            # YLE values usually define the leading edge points of sections.
            # For a simple wing defined from root to tip (e.g. 0 to B/2 for symmetric),
            # the y_coord for twist could be these yles.
            
            y_coords_normalized = np.abs(section_yles - section_yles[0]) / (np.abs(section_yles[-1] - section_yles[0]) if num_sections > 1 else 1.0)

            for i in range(num_sections):
                # Additive twist based on normalized position
                # This adds to any existing twist in initial_aincs
                twist_increment = root_twist_deg + (tip_twist_deg - root_twist_deg) * y_coords_normalized[i]
                new_aincs[i] = initial_aincs[i] + twist_increment 
        elif num_sections == 1: # Single section wing, apply average twist or tip_twist if that's the convention
            new_aincs[0] = initial_aincs[0] + (root_twist_deg + tip_twist_deg) / 2.0
        
        try:
            ovl_solver.set_surface_param(wing_name, "aincs", new_aincs, update_geom=True)
            # print(f"Updated aincs: {new_aincs}")
        except Exception as e:
            print(f"Error updating twist: {e}")

def update_span(ovl_solver, wing_name, span_factor, initial_yles, initial_span, initial_chords_for_sref_update):
    if ovl_solver and wing_name:
        new_span = initial_span * span_factor
        
        # Scaling yles:
        # If initial_yles = [0, Ytip_orig], then new_yles = [0, Ytip_orig * span_factor]
        # If initial_yles = [-Ytip_orig, 0, Ytip_orig], then scale non-zero parts.
        # For rectangle.avl, yles is often like [y_root, y_tip_half_span]
        # The scaling should preserve the relative distribution of sections.
        
        # Simple scaling from the root y-position (assuming it's the first yle)
        y_root = initial_yles[0]
        scaled_offsets = (initial_yles - y_root) * span_factor
        new_yles = y_root + scaled_offsets
        
        try:
            ovl_solver.set_surface_param(wing_name, "yles", new_yles, update_geom=True)
            # print(f"Updated yles: {new_yles}")
            # Update Bref and Sref
            ref_data = ovl_solver.get_reference_data()
            if 'Bref' in ref_data:
                ref_data['Bref'] = new_span
            if 'Sref' in ref_data and 'Cref' in ref_data: # Assuming Sref = Cref * Bref
                ref_data['Sref'] = ref_data['Cref'] * new_span
            elif 'Sref' in ref_data and len(initial_chords_for_sref_update)>0 : # Fallback if Cref not used, use root chord
                ref_data['Sref'] = initial_chords_for_sref_update[0] * new_span


            ovl_solver.set_reference_data(ref_data)

        except Exception as e:
            print(f"Error updating span/yles: {e}")

print("Helper functions defined.")

## Core Analysis and Plotting Function
This function, `run_and_plot`, will be connected to the interactive sliders. It will:
1. Update the aircraft geometry using the helper functions based on slider values.
2. Run an aerodynamic analysis using OptVL.
3. Plot the resulting lift distribution and a simple top-down view of the wing.

In [ ]:
# Global figure and axes objects for persistent plotting
fig = None
ax_lift = None
ax_geom = None

def setup_plots():
    global fig, ax_lift, ax_geom
    # Close previous figure if it exists to prevent overlaps in some environments
    if fig:
        plt.close(fig)
    
    fig, (ax_lift, ax_geom) = plt.subplots(2, 1, figsize=(10, 8))
    
    ax_lift.set_xlabel("Spanwise Location (Y LE)")
    ax_lift.set_ylabel("Lift Distribution (Cl * c / Cref)")
    ax_lift.set_title("Lift Distribution")
    ax_lift.grid(True)

    ax_geom.set_xlabel("Spanwise Location (Y)")
    ax_geom.set_ylabel("Chordwise Location (X)")
    ax_geom.set_title("Wing Geometry (Top-Down View)")
    ax_geom.axis('equal')
    ax_geom.grid(True)
    
    fig.tight_layout(pad=2.0)


def run_and_plot(chord_factor, tip_twist_deg, span_factor):
    global fig, ax_lift, ax_geom

    if not ovl or not main_wing_name:
        print("OptVL is not loaded or main wing name is not set. Cannot run analysis.")
        if not fig: # If plots not even set up, do it once
            setup_plots()
        ax_lift.clear()
        ax_geom.clear()
        ax_lift.text(0.5, 0.5, "OptVL not loaded", ha='center', va='center')
        ax_geom.text(0.5, 0.5, "OptVL not loaded", ha='center', va='center')
        plt.draw() # Make sure the cleared plot is shown
        return

    # Ensure plots are set up
    if fig is None or ax_lift is None or ax_geom is None or not plt.fignum_exists(fig.number):
         setup_plots()

    # 1. Update geometry
    # Assuming root_twist_deg is 0 for simplicity in this example for the slider
    # Pass original_span and original_yles to twist for normalization reference
    update_chord(ovl, main_wing_name, chord_factor, original_chords)
    update_twist(ovl, main_wing_name, tip_twist_deg, 0.0, original_aincs, original_yles, original_span)
    update_span(ovl, main_wing_name, span_factor, original_yles, original_span, original_chords)

    try:
        # 2. Set analysis parameters (can be fixed for this example)
        ovl.set_constraint("alpha", 5.0) 
        ovl.set_parameter("Mach", 0.1)
        # Add other necessary parameters if rectangle.avl requires them (e.g., density)
        # For basic AVL runs, Mach and alpha are often enough if density, velocity etc. have defaults or are not critical.
        # Check if parameters like density need to be set if not already in the AVL file or OVLSolver defaults
        current_density = ovl.get_parameter("density")
        if current_density == 0.0: # Or some other indicator of not being set
            ovl.set_parameter("density", 1.225)


        # 3. Execute OptVL run
        ovl.execute_run()

        # 4. Retrieve strip forces
        strip_forces = ovl.get_strip_forces()
        
        if main_wing_name not in strip_forces:
            print(f"Warning: '{main_wing_name}' not found in strip_forces output.")
            ax_lift.clear()
            ax_geom.clear()
            ax_lift.text(0.5, 0.5, f"Data for {main_wing_name} not found", ha='center', va='center')
            # Plot current geometry even if forces fail
            plot_current_geometry(ax_geom)
            plt.draw()
            return

        wing_data = strip_forces[main_wing_name]
        y_le = wing_data["Y LE"]
        lift_dist = wing_data["lift dist"] # This is Cl * c / Cref as per OVLSolver

        # 5. Plot results
        ax_lift.clear()
        ax_lift.plot(y_le, lift_dist, marker='o-', label=f"Alpha=5deg, M=0.1")
        ax_lift.set_xlabel("Spanwise Location (Y LE)")
        ax_lift.set_ylabel("Lift Distribution (Cl * c / Cref)")
        ax_lift.set_title("Lift Distribution")
        ax_lift.grid(True)
        ax_lift.legend()
        
        # Plot geometry
        plot_current_geometry(ax_geom)

    except Exception as e:
        print(f"An error occurred in run_and_plot: {e}")
        ax_lift.clear()
        ax_geom.clear()
        ax_lift.text(0.5, 0.5, f"Error: {e}", ha='center', va='center', wrap=True)
        ax_geom.text(0.5, 0.5, "Error during analysis/plotting", ha='center', va='center')

    # Redraw the full figure
    fig.canvas.draw_idle()
    # plt.show() # Not needed with %matplotlib widget or inline usually

def plot_current_geometry(ax_geom_to_plot_on):
    if not ovl or not main_wing_name:
        return
    try:
        current_params = ovl.get_surface_params(surf_name=main_wing_name, include_geom=True)
        xles = current_params[main_wing_name]['xles']
        yles = current_params[main_wing_name]['yles']
        chords = current_params[main_wing_name]['chords']
        
        ax_geom_to_plot_on.clear()
        # Plotting LE and TE
        ax_geom_to_plot_on.plot(yles, xles, 'bo-', label='Leading Edge')
        ax_geom_to_plot_on.plot(yles, xles - chords, 'ro-', label='Trailing Edge') # X decreases with chord typically

        # Connecting wingtips if multiple sections
        if len(yles) > 1:
            # Tip 1 (assuming first section is one tip/root)
            ax_geom_to_plot_on.plot([yles[0], yles[0]], [xles[0], xles[0]-chords[0]], 'k-')
            # Tip 2 (assuming last section is other tip/root)
            ax_geom_to_plot_on.plot([yles[-1], yles[-1]], [xles[-1], xles[-1]-chords[-1]], 'k-')

        ax_geom_to_plot_on.set_xlabel("Spanwise Location (Y)")
        ax_geom_to_plot_on.set_ylabel("Chordwise Location (X)")
        ax_geom_to_plot_on.set_title("Wing Geometry (Top-Down View)")
        ax_geom_to_plot_on.axis('equal')
        ax_geom_to_plot_on.grid(True)
        ax_geom_to_plot_on.legend()
    except Exception as e:
        print(f"Error plotting current geometry: {e}")
        ax_geom_to_plot_on.text(0.5, 0.5, "Error in geometry plot", ha='center', va='center')

# Call setup_plots once to initialize the figure and axes when this cell is run
setup_plots()
print("run_and_plot function defined and initial plot setup.")



## Interactive Controls
Use the sliders below to adjust the wing geometry parameters. The lift distribution and wing geometry plots will update automatically.
- **Chord Factor:** Multiplies the original chord length of all wing sections.
- **Tip Twist (deg):** Sets the twist angle at the wingtip. The twist varies linearly from 0 degrees at the root to this value at the tip (this is an *additional* twist applied on top of any existing twist in the AVL file).
- **Span Factor:** Multiplies the original wing span. The wing area will also change. Aspect ratio changes as a result of span and chord modifications.

In [ ]:
if ovl: # Only set up interact if OptVL loaded properly
    interact(
        run_and_plot, 
        chord_factor=FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='Chord Factor:'),
        tip_twist_deg=FloatSlider(min=-10.0, max=10.0, step=1.0, value=0.0, description='Tip Twist (deg):'),
        span_factor=FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='Span Factor:')
    )
    # Trigger an initial plot with default values
    #run_and_plot(chord_factor=1.0, tip_twist_deg=0.0, span_factor=1.0)
    # Display the figure explicitly if it doesn't show up automatically with interact in all environments
    # display(fig) # display(fig) is often needed if %matplotlib widget is used and interact doesn't show it.
                   # The setup_plots() and run_and_plot() should handle drawing.
else:
    print("OptVL not loaded. Interactive controls cannot be set up.")

# It's good practice to ensure the plot is visible after setting up interact.
# If the plot from the previous cell (run_and_plot definition cell) isn't visible, 
# this can help. However, %matplotlib widget should handle this.
if fig:
    display(fig) # This ensures the figure is shown after the interact setup.
                 # In some Jupyter environments, the plot might not show immediately
                 # after the cell defining run_and_plot is executed, but this will make it appear.

## Conclusion and Further Exploration

This notebook demonstrated how OptVL can be used interactively to explore the impact of geometric changes on aerodynamic performance, specifically the lift distribution. By using Python and `ipywidgets`, we created a simple tool for quick "what-if" scenarios.

**Further Exploration Ideas:**
-   Modify the range of the sliders or add new sliders for other parameters (e.g., root twist, angle of attack for the analysis).
-   Experiment with different AVL files for more complex aircraft geometries. You'll need to adjust the `main_wing_name` and potentially the geometry modification logic.
-   Plot other aerodynamic quantities available from `ovl.get_strip_forces()` or `ovl.get_total_forces()`.
-   Implement more sophisticated geometry changes (e.g., taper ratio, sweep angle).
-   Compare the results with theoretical predictions or other aerodynamic tools.